In [2]:
from ultralytics import YOLO
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# 집의 바운딩 박스 크기로 판별하는 함수
def house_size(results, checking_class): # 욜로 모델로 예측된 결과값과 각 객체들의 예측 클래스 번호가 담긴 리스트를 인자로 받음
    for result in results: # 검출된 객체들을 순회
        boxes = result.boxes # 각 객체의 바운딩박스 정보를 boxes로 정의

    houses =[] # 객체들이 담긴 리스트에서 집로 판별된 객체 리스트의 인덱스 번호를 담을 리스트
    xyxy = []
    bbox_sizes = [] # 바운딩 박스의 크기 정보를 담을 리스트

    for idx, data in enumerate(checking_class): # 탐지된 객체들의 클래스 정보가 담긴 리스트
        if data == 1: # 클래스 번호가 6일시 ("집"클래스의 번호가 1임)
            house_num = idx # 해당 리스트의 인덱스 번호를 trees리스트에 추가
            houses.append(house_num)
            
    print(houses)

    # 집로 검출된 객체들의 바운딩 박스 넓이를 구해서 리스트에 추가하는 작업
    for house_num in houses: # 집로 탐지된 객체의 인덱스 번호가 담긴 리스트를 순회
        if house_num != -1:
            house_xyxy_cordi = boxes.xyxy[house_num].tolist() # 해당 객체의 바운딩 박스의 좌상단 우하단 좌표
            bbox_size = (house_xyxy_cordi[2] - house_xyxy_cordi[0]) * (house_xyxy_cordi[3] - house_xyxy_cordi[1]) # 좌표를 통해 바운딩 박스의 가로 세로 길이를 구해 넓이를 구함
            bbox_sizes.append(bbox_size) # 구한 바운딩 박스 넓이를 리스트에 추가

    # 가장 큰 바운딩 박스의 인덱스 찾기
    largest_bbox_idx = bbox_sizes.index(max(bbox_sizes))

    # 가장 큰 바운딩 박스의 크기와 정보 출력
    largest_bbox_size = bbox_sizes[largest_bbox_idx]
    largest_bbox_info = house_xyxy_cordi[largest_bbox_idx]
    # 가장 큰 바운딩 박스의 인덱스 찾기
    largest_bbox_idx = bbox_sizes.index(max(bbox_sizes))

    print("가장 큰 바운딩 박스의 인덱스:", largest_bbox_idx)

    house_num = houses[largest_bbox_idx]
    house_num

    door_yn, loc, roof_yn, window_cnt, size = house_assess(house_num, results, checking_class)

    return door_yn, loc, roof_yn, window_cnt, size

In [4]:
# 검출된 객체간의 iou를 계산하는 함수
def cal_iou(boxA, boxB):
    
    # boxA와 boxB의 x, y 좌표 값 중 큰 값을 선택하여 IoU 계산에 사용할 시작 좌표를 결정
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    
    # boxA와 boxB의 x, y 좌표 값 중 작은 값을 선택하여 IoU 계산에 사용할 끝 좌표를 결정
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # 두 상자의 겹치는 영역의 넓이 계산
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # 각 상자의 전체 영역의 넓이 계산
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # iou계산
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [5]:
# 가장 큰 bbox 집의 인덱스 번호, 객체들 정보, 객체들의 클래스 리스트를 받아 각 요소들을 라벨링하는 함수
def house_assess(house_num, results, checking_class):
    for result in results: # 탐지된 객체들을 순회하면서
        boxes = result.boxes # 각 객체의 bbox정보를 boxes에 할당

    house_xyxy_cordi = boxes.xyxy[house_num].tolist() # 바운딩 박스의 좌상단 우하단 좌표 리스트
    house_center_x_cordi = boxes.xywh[house_num].tolist()[0] # 바운딩 박스의 중심좌표와 가로 세로 길이 리스트
    
    # 창문, 문, 지붕, 위치, 크기
    
    # 창문 ============================================================
    window_class_num = [] # 창문 클래스의 인덱스 번호를 담을 리스트
    for idx, data in enumerate(checking_class): # 객체들의 클래스 번호가 담긴 리스트를 순회하면서
        if data == 3 and boxes.conf[idx] > 0.6: # 리스트에 담긴 클래스 번호가 3이고 신뢰도가 0.6 초과일시
            window_class_num.append(idx) # 해당 인덱스 번호를 window_class_num에 추가
            
    window_xyxy_cordi = [] # 창문 바운딩 박스의 좌표 정보를 담을 리스트
    for idx in window_class_num: # 창문의 인덱스 번호가 담긴 리스트를 순회 하면서
        window_xyxy_cordi.append(boxes.xyxy[idx].tolist()) # 창문의 바운딩 박스 좌상단 우하단 좌표 리스트를 리스트에 추가
        
    window_iou = [] # 창문의 iou 결과값을 담을 리스트
    for i in window_xyxy_cordi: # 창문의 바운딩 박스 정보가 담긴 리스트를 순회
        window_iou.append(cal_iou(house_xyxy_cordi,i)) # 집 바운딩 박스와 창문 바운딩 박스의 iou를 계산한 값을 리스트에 추가
        
    window_count = 0 # 창문의 개수 초기화
    for i in window_iou: # 창문와 집의 iou값을 순회
        if i != 0: # iou값이 0이 아니면(iou가 0일시 집와 창문가 겹치지않음 -> 집로 판단된 영역안에 창문가 없기에 창문로 판단된 객체가 잘못 된것이라 예측해 카운팅X)
            window_count += 1 # 창문의 개수 +1
            
    window_cnt = '' # 창문 라벨링 정보 초기화
    if window_count == 0: # 창문의 카운팅수가 0일시 
        window_cnt = 'absence'
    elif window_count == 1 or window_count == 2: # 창문의 카운팅수가 1 or 2일시 
        window_cnt = '1 or 2'
    elif window_count >= 3:
        window_cnt = 'more than 3'
    
    # 위와 같은 과정으로 나머지 요소들도 iou를 바탕으로 집로 판정된 영역 내에 겹치지 않는 것을 제외하고 카운팅해 라벨링
    
    # 지붕 ============================================================
    roof_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 2:
            roof_class_num.append(idx)
    roof_xyxy_cordi = []
    for idx in roof_class_num:
        roof_xyxy_cordi.append(boxes.xyxy[idx].tolist())
    roof_xyxy_cordi
    roof_iou = []
    for i in roof_xyxy_cordi:
        roof_iou.append(cal_iou(house_xyxy_cordi,i))
    roof_count = 0
    for i in roof_iou:
        if i != 0:
            roof_count += 1
    roof_yn = ''
    if roof_count == 0:
        roof_yn = 'n'
    else:
        roof_yn = 'y'
    
    # 문 ============================================================
    door_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 0 and boxes.conf[idx] > 0.4:
            door_class_num.append(idx)
    door_xyxy_cordi = []
    for idx in door_class_num:
        door_xyxy_cordi.append(boxes.xyxy[idx].tolist())
    door_iou = []
    for i in door_xyxy_cordi:
        door_iou.append(cal_iou(house_xyxy_cordi,i))
    door_count = 0
    for i in door_iou:
        if i != 0:
            door_count += 1
    door_yn = ''
    if door_count == 0:
        door_yn = 'n'
    else:
        door_yn = 'y'
    
    # 위치와 크기 ============================================================
    image_w = boxes.orig_shape[0]*0.965 # 입력받은 이미지 width
    image_h = boxes.orig_shape[1] # 입력받은 이미지 height
    image_size = image_w * image_h # 입력받은 이미지 넓이
    house_size = (house_xyxy_cordi[2] - house_xyxy_cordi[0]) * (house_xyxy_cordi[3] - house_xyxy_cordi[1]) # 집 bbox의 넓이
    house_size_percent = house_size / image_size * 100 # 입력받은 이미지 대비 집의 bbox가 몇% 차지하는지 계산
    
    # 크기
    size = '' # 크기 라벨링 정보 초기화
    if house_size_percent < 15*0.8:
        size = 'small'
    elif house_size_percent >= 15*0.8 and house_size_percent < 50*0.8:
        size = 'middle'
    elif house_size_percent >= 50*0.8:
        size = 'big'
    
    # 위치
    loc = '' # 위치 라벨링 정보 초기화
    if house_center_x_cordi < image_w/2.7: 
        loc = 'left'
    elif house_center_x_cordi >= image_w/2.7 and house_center_x_cordi < image_w/3*2:
        loc = 'center'
    elif house_center_x_cordi >= image_w/3*2:
        loc = 'right'
    
    return door_yn, loc, roof_yn, window_cnt, size

In [6]:
# 나무로 탐지된 객체수에 따라 라벨링을 결정
def house_csv(results): # 객체들의 정보를 인자로 받음
    for result in results: # 객체들을 순회하면서
        boxes = result.boxes # 각 객체의 bbox 정보를 boxes로 할당
        class_names = result.names # 각 객체의 클래스이름을 담음
        
    checking_class = boxes.cls.int().tolist() # 클래스 번호를 리스트로 담음
    checking_conf = boxes.conf.tolist() # bbox의 신뢰도를 리스트로 담음

    house_num = -1
    
    for idx, data in enumerate(checking_class):
        if data == 1:
            house_num = idx
            break
    
    if house_num != -1:
        if checking_class.count(1) >= 2: # 객체들의 클래스 번호가 담긴 리스트에서 1이 2개 이상일시
            print('집이 2개 이상 검출되었습니다.')
            # 크기가 가장 큰 바운딩 박스 찾는 함수 호출
            door_yn, loc, roof_yn, window_cnt, size = house_size(results, checking_class)
        elif checking_class.count(1) == 0: # 객체들의 클래스 번호가 담긴 리스트에서 1이 1개 일시
            print('집이 검출되지 않았습니다.')
        else:
            print("집이 1개 검출되었습니다.")
            # 예측하는 함수 호출
            door_yn, loc, roof_yn, window_cnt, size = house_size(results, checking_class)

    else:
        door_yn = 'no detect'
        roof_yn = 'no detect'
        loc = 'no detect'
        window_cnt = 'no detect'
        size = 'no detect'

    return door_yn, loc, roof_yn, window_cnt, size

In [7]:
model = YOLO('./house_x_weight/weights/best.pt')

In [8]:
def yolov8_house_detect(path,filename):
    
    results = model.predict(path+ '/' + filename)
    
    res_plot = results[0].plot()
    
    plt.imshow(res_plot)

    door_yn, loc, roof_yn, window_cnt, size = house_csv(results)
    
    return door_yn, loc, roof_yn, window_cnt, size

In [9]:
def house_save(img_dir, save_dir):
    
    all_images = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f)) and f.endswith(('.png', '.jpg', '.jpeg'))]

    filenames = []

    house_df = pd.DataFrame(columns=('id', 'door_yn', 'loc', 'roof_yn', 'window_cnt', 'size'))

    for i in all_images:

        filenames.append(i.split('.')[0][:-4])

    for filename, i in tqdm(zip(all_images, filenames)):
        door_yn, loc, roof_yn, window_cnt, size= yolov8_house_detect(img_dir, filename)
        output_filename =save_dir + i + '.jpg'
        plt.savefig(output_filename, format='jpg', dpi=300)
        plt.close()
        
        house_df.loc[len(house_df)] = [i, door_yn, loc, roof_yn, window_cnt, size]
    
    house_df.to_csv(save_dir +'house_result2.csv', index=False)

In [10]:
house_save(img_dir='./house.v1i.yolov8/test/images', save_dir= './house_result/')

0it [00:00, ?it/s]
image 1/1 c:\yolov8\house.v1i.yolov8\test\images\24_175_23019_house_jpg.rf.109cd1c2a18292112be3c3083dbd72f2.jpg: 640x640 1 house, 1 roof, 53.0ms
Speed: 4.0ms preprocess, 53.0ms inference, 23.6ms postprocess per image at shape (1, 3, 640, 640)
1it [00:08,  8.85s/it]

집이 1개 검출되었습니다.
[1]
가장 큰 바운딩 박스의 인덱스: 0



image 1/1 c:\yolov8\house.v1i.yolov8\test\images\24_176_23025_house_jpg.rf.7e0641d53fc0b1988ba93882132d339d.jpg: 640x640 1 house, 1 roof, 2 windows, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
2it [00:09,  3.78s/it]


집이 1개 검출되었습니다.
[1]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\house.v1i.yolov8\test\images\24_176_23035_house_jpg.rf.69e6dbdc934c2898ac88ba7080bd6d50.jpg: 640x640 1 door, 1 house, 50.0ms
Speed: 2.0ms preprocess, 50.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
3it [00:09,  2.16s/it]


...

집이 1개 검출되었습니다.
[3]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\house.v1i.yolov8\test\images\5_28_23077_house_jpg.rf.c3cdfc7906e9bfa2f0e19f3f55f4998e.jpg: 640x640 1 door, 1 house, 1 roof, 1 window, 56.0ms
Speed: 2.0ms preprocess, 56.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
99it [00:32,  4.24it/s]


집이 1개 검출되었습니다.
[1]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\house.v1i.yolov8\test\images\5_28_23081_house_jpg.rf.7155c549690a9a8e586c72210241cdca.jpg: 640x640 1 door, 1 house, 1 roof, 1 window, 57.0ms
Speed: 1.0ms preprocess, 57.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
100it [00:32,  3.05it/s]

집이 1개 검출되었습니다.
[1]
가장 큰 바운딩 박스의 인덱스: 0


In [11]:
house_ori_df = pd.read_csv('./dbi_house.csv')
house_pred_df = pd.read_csv('./house_result/house_result2.csv')
house_ori_100_df = house_ori_df[house_ori_df['id'].isin(house_pred_df['id'])]

door_count = 0
loc_count = 0
roof_count = 0
window_count = 0
size_count = 0
for i in range(len(house_pred_df)):
    if house_ori_100_df['door_yn'].tolist()[i] != house_pred_df['door_yn'].tolist()[i]:
        door_count += 1
    if house_ori_100_df['loc'].tolist()[i] != house_pred_df['loc'].tolist()[i]:
        loc_count += 1
    if house_ori_100_df['roof_yn'].tolist()[i] != house_pred_df['roof_yn'].tolist()[i]:
        roof_count += 1
    if house_ori_100_df['window_cnt'].tolist()[i] != house_pred_df['window_cnt'].tolist()[i]:
        window_count += 1
    if house_ori_100_df['size'].tolist()[i] != house_pred_df['size'].tolist()[i]:
        size_count += 1
print(f'door diff : {door_count}')
print(f'loc diff : {loc_count}')
print(f'roof diff : {roof_count}')
print(f'window diff : {window_count}')
print(f'size diff : {size_count}')

door diff : 7
loc diff : 7
roof diff : 10
window diff : 14
size diff : 7


In [12]:
window_wrong = []
for i in range(len(house_pred_df)): 
    if house_ori_100_df['window_cnt'].iloc[i] != house_pred_df['window_cnt'].iloc[i]:
        window_wrong.append([house_pred_df['id'].iloc[i], house_ori_100_df['window_cnt'].iloc[i], house_pred_df['window_cnt'].iloc[i]])

print(len(window_wrong))

14


In [13]:
for i in window_wrong:
    print(i)

['24_176_23035_house', '1 or 2', 'absence']
['24_178_23067_house', '1 or 2', 'more than 3']
['24_179_23107_house', '1 or 2', 'more than 3']
['26_139_23017_house', '1 or 2', 'more than 3']
['26_140_23027_house', 'more than 3', 'absence']
['33_95_23089_house', '1 or 2', 'absence']
['33_97_23149_house', '1 or 2', 'more than 3']
['33_97_23153_house', 'more than 3', 'absence']
['3_12_23095_house', '1 or 2', 'absence']
['3_13_23111_house', '1 or 2', 'absence']
['50_229_23061_house', '1 or 2', 'absence']
['50_230_23084_house', 'more than 3', 'absence']
['50_231_23107_house', '1 or 2', 'absence']
['5_26_23007_house', 'more than 3', 'absence']


In [14]:
door_wrong = []
for i in range(len(house_pred_df)): 
    if house_ori_100_df['door_yn'].iloc[i] != house_pred_df['door_yn'].iloc[i]:
        door_wrong.append([house_pred_df['id'].iloc[i], house_ori_100_df['door_yn'].iloc[i], house_pred_df['door_yn'].iloc[i]])

print(len(door_wrong))

7


In [15]:
for i in door_wrong:
    print(i)

['24_180_23138_house', 'y', 'n']
['33_95_23089_house', 'n', 'y']
['33_96_23116_house', 'y', 'n']
['33_97_23149_house', 'y', 'n']
['38_105_23025_house', 'y', 'n']
['3_13_23111_house', 'n', 'y']
['50_230_23088_house', 'n', 'y']


In [16]:
roof_wrong = []
for i in range(len(house_pred_df)): 
    if house_ori_100_df['roof_yn'].iloc[i] != house_pred_df['roof_yn'].iloc[i]:
        roof_wrong.append([house_pred_df['id'].iloc[i], house_ori_100_df['roof_yn'].iloc[i], house_pred_df['roof_yn'].iloc[i]])

print(len(roof_wrong))

10


In [17]:
for i in roof_wrong:
    print(i)

['24_175_23019_house', 'n', 'y']
['26_140_23027_house', 'y', 'n']
['33_95_23089_house', 'y', 'n']
['38_106_23062_house', 'y', 'n']
['3_11_23079_house', 'y', 'n']
['3_13_23111_house', 'y', 'n']
['50_229_23061_house', 'n', 'y']
['50_229_23070_house', 'y', 'n']
['52_218_23119_house', 'y', 'n']
['5_26_23007_house', 'y', 'n']


In [18]:
loc_wrong = []
for i in range(len(house_pred_df)): 
    if house_ori_100_df['loc'].iloc[i] != house_pred_df['loc'].iloc[i]:
        loc_wrong.append([house_pred_df['id'].iloc[i], house_ori_100_df['loc'].iloc[i], house_pred_df['loc'].iloc[i]])

print(len(loc_wrong))

7


In [19]:
for i in loc_wrong:
    print(i)

['24_177_23043_house', 'left', 'center']
['24_178_23087_house', 'center', 'left']
['33_97_23149_house', 'center', 'left']
['33_98_23178_house', 'right', 'center']
['38_107_23069_house', 'center', 'left']
['47_197_23004_house', 'left', 'center']
['52_217_23071_house', 'left', 'center']


In [20]:
size_wrong = []
for i in range(len(house_pred_df)): 
    if house_ori_100_df['size'].iloc[i] != house_pred_df['size'].iloc[i]:
        size_wrong.append([house_pred_df['id'].iloc[i], house_ori_100_df['size'].iloc[i], house_pred_df['size'].iloc[i]])

print(len(size_wrong))

7


In [21]:
for i in size_wrong:
    print(i)

['24_180_23121_house', 'small', 'middle']
['26_139_23002_house', 'small', 'middle']
['33_93_23057_house', 'middle', 'big']
['33_98_23194_house', 'small', 'middle']
['50_230_23084_house', 'big', 'middle']
['52_214_23003_house', 'middle', 'small']
['5_28_23077_house', 'small', 'middle']
